In [1]:
#Осторожно - черновик

import os, sys
import csv
import time
from pprint import pprint

import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize

sys.path.append("./app/api")
sys.path.append("./app/recommendation")

from utils.parse_big_json import extract_json

In [2]:
DATA_SET_PATH = "./app/recommendation/Dataset/datasets_biblioteki/datasets_2/"

In [3]:
def sort_files(file_name):
    _id = 0
    file_name = os.path.splitext(file_name)[0]
    split_name = file_name.split("_")
    
    if len(split_name) > 1 and split_name[-1].isdigit():
        _id = int(split_name[-1])
        
    return _id

In [4]:
def search_files(prefix, suffix, path: str) -> list[str]:            
    files = [file_name for file_name in os.listdir(path) if file_name.startswith(prefix) and file_name.endswith(suffix)]
            
    files.sort(key=sort_files)
    return files

In [8]:
user_df = pd.read_csv(f"{DATA_SET_PATH}/readers.csv", delimiter=";", low_memory=False, encoding='cp1251', usecols=range(3), header=None)
user_df

,0,1,2
0,366670,14.10.1974,"Обручевский, 10"
1,175006,20.03.1978,"г. Москва, ул. Саранская, д. 7,"
2,334805,03.12.1985,"г. Москва, ул. Ясеневая, д. 12, корп. 1,"
3,364422,15.03.1992,"г. Владивосток, ул. Анны Щетининой, д. 20"
4,37574,18.11.1981,"г. Москва, ул. Менжинского, д. 28, корп. 2,"
...,...,...,...
569817,461102,10.02.1978,"МО, Совхоз им. Ленина, 25-387"
569818,388950,07.02.1994,"г. Москва, ул. Мартеновская, д. 6, корп. 2,"
569819,719163,22.10.2000,"Свердловская обл., г. Нижний Тагил, ул. Красно..."
569820,222451,15.05.1983,"г. Москва, ул. Базовская, д. 15, корп. 8,"


In [8]:
gen = extract_json("./app/recommendation/Dataset/books_full.jsn")
# print(next(gen))

In [9]:
from core.deps import get_db
from schemas.book import BookCreate
from crud.crud_books import books_service

In [24]:
# 
collapse_field = {}
collapsed_ref_ids = {}
books = []
rubrics_map = {}
rubrics = []

for book in gen:
    smart_collapse_field = book.get("smart_collapse_field")
    if not smart_collapse_field:
        books.append({
            "id": book["id"],
            "title": book["title"],
            "author": book["author_fullNameAlt"],
            "rubric_id": book["rubric_id"],
        })
    elif not collapse_field.get(smart_collapse_field):
        collapse_field[smart_collapse_field] = book["id"]
        books.append({
            "id": book["id"],
            "title": book["title"],
            "author": book["author_fullNameAlt"],
            "rubric_id": book["rubric_id"],
        })
        
        if book.get("rubric_id"):
            if not rubrics_map.get(book["rubric_id"]):
                rubrics_map[book["rubric_id"]] = []
            rubrics_map[book["rubric_id"]].append(book["id"])
        
    collapsed_ref_ids[book["id"]] = collapse_field[smart_collapse_field]
      
print("books: ", len(books)) 
print("rubrics: ", len(rubrics)) 
print("collapse_field: ", len(collapse_field)) 
print("collapsed_ref_ids: ", len(collapsed_ref_ids)) 

books:  0
rubrics:  0
collapse_field:  0
collapsed_ref_ids:  0


In [25]:
for _key in rubrics_map:
    rubrics.append({
        "id": _key,
        "books": rubrics_map[_key]
    })

In [26]:
class CollectionHelper:
    def __init__(self, collection: list):
        self._collection = collection
        self._collection_ref = {}
        self._indexes = {}
        self.create_indexes()
        self.length = len(self._collection)
        
    def create_indexes(self):
        for i, item in enumerate(self._collection):
            self._indexes[item["id"]] = i
            
    def get_index(self, item_id: int) -> int:
        return self._indexes.get(item_id)
    
    def get_book_from_book_id(self, item_id: int) -> dict:
        index = self._indexes.get(item_id, None)
        if index is not None:
            return self._collection[index]
        
        return None
    
    def get_book_from_index(self, index: int) -> dict:
        if len(self._collection) - 1 > index:
            return self._collection[index]
    
books_helper = CollectionHelper(books)
rubrics_helper = CollectionHelper(rubrics)

In [27]:
circulation_files = search_files("circulaton", "csv", DATA_SET_PATH)
circulation_data_set = pd.concat((data_set for data_set in map(lambda f: pd.read_csv(f"{BASE_PATH}/{f}", delimiter=";", low_memory=False, encoding='cp1251', usecols=range(8)), circulation_files)))

NameError: name 'BASE_PATH' is not defined

In [17]:
import json
with open("./app/recommendation/data/books.json", "w") as f:
    f.write(json.dumps(books))
    
with open("./app/recommendation/data/rubrics.json", "w") as f:
    f.write(json.dumps(rubrics))
    
with open("./app/recommendation/data/collapse_field.json", "w") as f:
    f.write(json.dumps(collapse_field))
    
with open("./app/recommendation/data/collapsed_ref_ids.json", "w") as f:
    f.write(json.dumps(collapsed_ref_ids))

In [23]:
circulation_files = search_files("circulaton", "csv", DATA_SET_PATH)
# circulation_data_set = pd.concat((data_set for data_set in map(lambda f: pd.read_csv(f"{BASE_PATH}/{f}", delimiter=";", low_memory=False, encoding='cp1251', usecols=range(8)), circulation_files)))

users_read_map = {}
users_read_book_map = {}
users_associated = {}


for f in circulation_files:
    dataset = pd.read_csv(f"{DATA_SET_PATH}/{f}", delimiter=";", low_memory=False, encoding='cp1251', usecols=range(8))

    for i, data_row in enumerate(dataset.values):
        if not users_read_map.get(data_row[5]):
            users_read_map[data_row[5]] = []
            users_read_book_map[data_row[5]] = set()
            
        if data_row[1] in users_read_book_map[data_row[5]]:
            continue
            
        users_read_book_map[data_row[5]].add(data_row[1])
        
        book_id = collapsed_ref_ids.get(data_row[1])
        if data_row[5] == 582236:
            print(">>> ", data_row[1])
            
        if book_id:
            users_read_map[data_row[5]].append(book_id)  
        
for idx, user_id in enumerate(users_read_map):
    users_associated[idx] = user_id
    


>>>  505628
>>>  1190431
>>>  455012
>>>  1594092
>>>  247554
>>>  439242
>>>  264189
>>>  363371


In [ ]:
books_helper.get

In [20]:
user_id_search = 582236
print(users_read_map[user_id_search])

[505628, 505628, 505628, 1190431, 1190431, 1197833, 1884042, 1692399, 1712782, 264189, 1597791, 1597791, 1597791]


In [11]:
from collections import defaultdict

counter = defaultdict(int)

for user_id in users_read_map:
    for book_id in users_read_map[user_id]:
        counter[book_id] += 1

In [40]:
books_keys = list(counter.keys())
books_keys.sort(key=lambda x: counter[x], reverse=True)

print(counter[books_keys[0]])

15944


In [12]:
def make_coo_row(read_books: list, books_indexes_helper: CollectionHelper):
    idx = []
    values = []
    
    items = []
    
    for book_id in read_books:
        book = books_indexes_helper.get_book_from_book_id(book_id)
        if not book: 
            continue
        idx.append(books_indexes_helper.get_index(book["id"]))
        values.append(1.0)
        
    return sparse.coo_matrix(
        (np.array(values).astype(np.float32), ([0] * len(idx), idx)), shape=(1, books_indexes_helper.length)
    )
            
    

In [13]:
rows = []
for user_id in users_read_map:
    rows.append(make_coo_row(users_read_map[user_id], books_helper))

In [15]:
X_sparse = sparse.vstack(rows).tocsr()

In [14]:
X_sparse.shape
# del(rows)

(548426, 1595278)

In [28]:
for i, row in enumerate(X_sparse.T):
    print(row.sum(), row.tocsr())
    if i > 10:
        break

0.0 
12.0   (0, 23607)	2.0
  (0, 41154)	1.0
  (0, 55763)	1.0
  (0, 113637)	1.0
  (0, 156855)	1.0
  (0, 159173)	1.0
  (0, 164314)	1.0
  (0, 195574)	1.0
  (0, 354112)	3.0
0.0 
0.0 
1.0   (0, 130882)	1.0
3.0   (0, 153735)	1.0
  (0, 310354)	1.0
  (0, 450317)	1.0
0.0 
0.0 
144.0   (0, 1306)	2.0
  (0, 1497)	1.0
  (0, 4532)	1.0
  (0, 11825)	1.0
  (0, 14098)	2.0
  (0, 14784)	1.0
  (0, 17782)	1.0
  (0, 19703)	1.0
  (0, 21864)	1.0
  (0, 25212)	1.0
  (0, 37128)	3.0
  (0, 42265)	2.0
  (0, 53069)	1.0
  (0, 101827)	1.0
  (0, 112165)	2.0
  (0, 114267)	1.0
  (0, 141910)	1.0
  (0, 156511)	2.0
  (0, 158579)	1.0
  (0, 174656)	1.0
  (0, 176614)	1.0
  (0, 183471)	1.0
  (0, 184051)	1.0
  (0, 188558)	2.0
  (0, 188743)	4.0
  :	:
  (0, 362666)	1.0
  (0, 362704)	1.0
  (0, 364537)	1.0
  (0, 370785)	1.0
  (0, 370786)	1.0
  (0, 374066)	2.0
  (0, 375812)	1.0
  (0, 379130)	1.0
  (0, 380087)	1.0
  (0, 385535)	1.0
  (0, 385540)	1.0
  (0, 385546)	1.0
  (0, 385765)	1.0
  (0, 401236)	1.0
  (0, 402424)	2.0
  (0, 402839)	1

In [8]:
X_sparse_transposed = X_sparse.transpose(copy=False)
# sim2 = X_sparse * X_sparse_transposed
print(X_sparse_transposed.shape)

NameError: name 'X_sparse' is not defined

In [51]:
# fit = sim2 * X_sparse
# del(sim2)
# del(X_sparse_transposed)

In [55]:
format_book = lambda b: f"{b['title']}, {b['autor']}"

In [23]:
# item to item

# print(format_book(catalog_map[111000]))
# pprint([f"{format_book(catalog_map[catalog_associated[i + 1]])}, {catalog_associated[i + 1]}" for i in sim[7714].toarray().argsort()[0][-20:]])

def valid_result(book_id: int, tail: int = 20) -> int:
    t = time.time()
    book_target = catalog_map[book_id]
    print()
    print(format_book(book_target))
    result = sim[book_target["idx"]].toarray().argsort()[0][-20:]
#     print(result)
    for col_id in result:
        i = col_id
        associated = catalog_associated[i]
        print(f"    {associated}: {format_book(catalog_map[associated])}")
        
    return time.time() - t

def test():
    ids = [499858]
    for _id in ids:
        valid_result(_id)
        
# test()

In [17]:
from sklearn.decomposition import TruncatedSVD

n_colls = 500

In [26]:
svd = TruncatedSVD(n_components=n_colls)
X_dense = svd.fit_transform(X_sparse)

In [27]:
print(X_sparse.shape)
print(X_dense.shape)

(548426, 1595278)
(548426, 500)


In [33]:
X_sparse.getrow(548425)

<1x1595278 sparse matrix of type '<class 'numpy.float32'>'
	with 1 stored elements in Compressed Sparse Row format>

In [26]:
# knn
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=n_colls, metric="cosine", n_jobs=-1)
knn.fit(X_sparse)

NearestNeighbors(metric='cosine', n_jobs=1, n_neighbors=500)

In [ ]:
for row_sparse in rows:
    row_dense = svd.transform(row_sparse)

In [28]:
import time

# user to item
def search_idx_from_user_id(user_id: int) -> int:
    for idx in users_associated:
        if users_associated[idx] == user_id:
            return idx
    
def valid_user_to_item():
    users = [111000, 520605, 452856]
    
    for user_id in users:
        idx = search_idx_from_user_id(user_id)
        
        if not idx: continue

        row = rows[idx]
        t = time.time()
        knn_result = knn.kneighbors(row, n_neighbors=500)
        neighbors = knn_result[1]
        scores = np.asarray(X_sparse.tocsr()[neighbors[0]].sum(axis=0)[0]).flatten()
        top_indices = np.argsort(-scores)
#         print(len(top_indices))
        recommended_items = top_indices[:10]
        print("=" * 80)
        print("time: ", time.time() - t)
        print(user_id, " >> ", recommended_items)            
        
       
valid_user_to_item()

time:  0.30048704147338867
111000  >>  [1594467  469296 1229119  878094  405221  825486 1200410 1594469  471943
 1248440]
time:  0.28186464309692383
520605  >>  [ 875707 1594255  472177  471632 1242108 1247583  470233  875735 1594665
  875562]
time:  0.2835257053375244
452856  >>  [1247821  471814  878451 1247953  471815  878452  878546 1589896  472048
  875702]


In [26]:
X_dense_transposed = X_dense.transpose()
fit = X_dense * X_dense_transposed * X_dense

ValueError: operands could not be broadcast together with shapes (548426,500) (500,548426) 

In [61]:
fit.shape

(548426, 500)

In [70]:
fit[1520].argsort()[0]

173

In [20]:
import faiss

In [71]:
index = faiss.index_factory(500, "IVF256,PQ32", faiss.METRIC_INNER_PRODUCT)
index.train(X_sparse)

RuntimeError: Error in void faiss::ProductQuantizer::set_derived_values() at /__w/faiss-wheels/faiss-wheels/faiss/faiss/impl/ProductQuantizer.cpp:184: Error: 'd % M == 0' failed: The dimension of the vector (d) should be a multiple of the number of subquantizers (M)

In [22]:
sparse.save_npz('sparse_matrix.npz', X_sparse)